This notebook explores identifying multiword expressions using the part-of-speech filtering technique of Justeson and Katz (1995), "[Technical terminology: some linguistic properties and an algorithm for identification in text](https://brenocon.com/JustesonKatz1995.pdf)".

In [1]:
import spacy, re
from collections import Counter

In [2]:
nlp = spacy.load('en_core_web_sm', disable=['ner,parser'])
nlp.remove_pipe('ner')
nlp.remove_pipe('parser')

('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7ff0382d6ee0>)

In [3]:
def getTokens(filename):
    
    """ Read the first 1000 lines of an input file """
    tokens=[]
    with open(filename) as file:
        for idx,line in enumerate(file):
            tokens.extend(nlp(line))
            if idx > 1000:
                break
    return tokens

In [14]:
tokens=getTokens("../data/odyssey.txt")
print(len(tokens))

13815


In [15]:
words=[x.text for x in tokens]

Let's simplify the POS tags to make the regex easier to understand.

In [16]:
adjectives=set(["JJ", "JJR", "JJS"])
nouns=set(["NN", "NNS", "NNP", "NNPS"])

taglist=[]
for x in tokens:
    if x.tag_ in adjectives:
        taglist.append("ADJ")
    elif x.tag_ in nouns:
        taglist.append("NOUN")
    elif x.tag == "IN":
        taglist.append("PREP")
    else:
        taglist.append("O")
                
tags=' '.join(taglist)        

In [17]:
def getChar2TokenMap(tags):
    
    """  We'll search over the postag sequence, so we need to get the token ID for any
    character to be able to match the word token. """
    
    ws=re.compile(" ")
    char2token={}

    lastStart=0
    for idx, m in enumerate(ws.finditer(tags)):
        char2token[lastStart]=idx
        lastStart=m.start()+1
        
    return char2token

def getToken(tokenId, char2token):
    
    """ Find the token ID for given character in the POS sequence """
    while(tokenId > 0):
        if tokenId in char2token:
            return char2token[tokenId]
        tokenId-=1
    return None

In [18]:
char2token=getChar2TokenMap(tags)

Now let's find all sequences of POS tags that match the Justeson and Katz pattern of `(((ADJ|NOUN) )+|((ADJ|NOUN) )*(NOUN PREP )((ADJ|NOUN) )*)NOUN`

"In words, a candidate term is a multi-word noun phrase; and it either is a string of nouns and/or adjectives, ending in a noun, or it consists of two such strings, separated by a single preposition." (JK 17)

In [19]:
p = re.compile("(((ADJ|NOUN) )+|((ADJ|NOUN) )*(NOUN PREP )((ADJ|NOUN) )*)NOUN")

mweCount=Counter()

for m in p.finditer(tags):
    startToken=getToken(m.start(),char2token)
    endToken=getToken(m.end(),char2token)
    mwe=' '.join(words[startToken:endToken+1])
    mweCount[mwe]+=1

for k,v in mweCount.most_common(100):
    print(k,v)

own house 5
other hand 4
marriage gifts 3
barley meal 3
own people 2
other gods 2
girt island 2
poor man 2
immortal gods 2
dear father 2
old woman 2
funeral rites 2
due pomp 2
sad tale 2
old friend 2
outer court 2
good old woman 2
other women 2
public moment 2
excellent person 2
old man 2
leathern bags 2
bad end 2
false Aegisthus 2
MINERVA ’S VISIT 1
ingenious hero 1
famous town 1
Many cities 1
own life 1
own sheer folly 1
god Hyperion 1
whatsoever source 1
goddess Calypso 1
large cave 1
other East.1 1
Olympian Jove 1
own folly 1
good will 1
lonely sea 1
magician Atlas 1
great columns 1
poor unhappy Ulysses 1
own chimneys 1
capable man 1
Polyphemus king 1
nymph Thoosa 1
king Phorcys 1
Ogygian island 1
son Telemachus 1
golden sandals 1
redoubtable bronze 1
shod spear 1
topmost summits 1
bronze spear 1
lordly suitors 1
wet sponges 1
great quantities 1
brave father 1
right hand 1
many other spears 1
unhappy father 1
maid servant 1
beautiful golden ewer 1
silver basin 1
clean table 1
upper

We'll define our MWE dictionary to be the 1000 most frequent sequences matched.

In [20]:
my_mwe=[k for (k,v) in mweCount.most_common(1000)]

Now let's transform each MWE into a single token (e.g., replace `New York City` with `New_York_City`)

In [21]:
def replaceMWE(text, mweList):
    
    """ Replace all instances of MWEs in text with single token 
    
    MWEs are ranked from longest to shortest so that longest replacements are made first (e.g.,
    "New York City" is matched first before "New York")
    
    """
    
    sorted_by_length = sorted(mweList, key=len, reverse=True)
    for mwe in sorted_by_length:
        text=re.sub(re.escape(mwe), re.sub(" ", "_", mwe), text)
    return text

In [24]:
processedText=replaceMWE("The poor man, is one who is a dear father", my_mwe)

In [25]:
print(processedText)

The poor_man, is one who is a dear_father


Q1. Plug in your own data into `getTokens` above and identify the MWE it contains.  How do you think MWE would perform for your classification task?